# simple GenAI app

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")

In [2]:
# data ingestion
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader(web_path="https://python.langchain.com/docs/versions/v0_3/")
doc=loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
len(doc)

1

In [4]:
# text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
split_doc=text_splitter.split_documents(doc)

In [5]:
len(split_doc)

44

In [6]:
# embedding and vector store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small"
)

db=FAISS.from_documents(split_doc,embeddings)
db

In [7]:
# query from a vector store db
query="Whats changes in v0.3"
result=db.similarity_search(query)
result

[Document(id='96ffd351-40ce-42d0-9f63-93af76c83b8a', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content="Last updated: 09.16.24\nWhat's changed\u200b"),
 Document(id='29dc4041-c432-4494-b15b-9fa130d8be90', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='Added utilities for interacting with chat models: universal model constructor, rate limiter, message utilities,\nAdded the ability to dispatch custom events.\nRevamped integration docs and API reference. Read more here.\nMarked as deprecated a number of legacy chains and added migration guides for all of them. These are slated for removal in langchain 1.0.0. See the deprecated chains and associated migration guides here.'),
 Document(id='c43ab518-3e1a-4497-9522-19

In [8]:
result[0].page_content

"Last updated: 09.16.24\nWhat's changed\u200b"

In [9]:
result[1].page_content

'Added utilities for interacting with chat models: universal model constructor, rate limiter, message utilities,\nAdded the ability to dispatch custom events.\nRevamped integration docs and API reference. Read more here.\nMarked as deprecated a number of legacy chains and added migration guides for all of them. These are slated for removal in langchain 1.0.0. See the deprecated chains and associated migration guides here.'

In [11]:
from langchain_openai import ChatOpenAI


llm=ChatOpenAI()

In [ ]:
## Retrieval Chain, Document Chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """Answer the follwing question  based only on provided context:
    <context> 
    {context}
    </context>
    queston: {input}
    """
)

document_chain= create_stuff_documents_chain(llm,prompt)
 # will responsible to provide context to the prompt
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the follwing question {input} based only on provided context:\n    <context> \n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002BDD41C8830>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002BDD41C8AD0>, root_client=<openai.OpenAI object at 0x000002BDD40CED50>, root_async_client=<openai.AsyncOpenAI object at 0x000002BDD40CF390>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwarg

In [32]:
document_chain.invoke({
    "context":result
})


KeyError: "Input to ChatPromptTemplate is missing variables {'input'}.  Expected: ['context', 'input'] Received: ['context']\nNote: if you intended {input} to be part of the string and not a variable, please escape it with double curly braces like: '{{input}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [33]:
# Retrieval Chain
# input-> Retriever-->vectorstore->similarity search->document
retriever=db.as_retriever()

In [34]:
from langchain.chains import create_retrieval_chain

retriever_chain=create_retrieval_chain(retriever,document_chain)

In [35]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002BD4F8CD7F0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the follwing question {input} based only on provided context:\n    <context> \n    {context}\n    </context>\n    '), additi

In [36]:
## Get the response from the LLM
response=retriever_chain.invoke({
    "input":"what is the latest version of langchain and what improvements are in it"
})
response['answer']

'The latest version of LangChain is version 0.3. Some improvements in this version include the ability to automatically update deprecated imports using the langchain-cli, compatibility with both langchain-core 0.2 and 0.3, and minor version bumps for packages that now require langchain-core 0.3.'

In [37]:
response

{'input': 'what is the latest version of langchain and what improvements are in it',
 'context': [Document(id='111ca602-0e2d-4ffe-bc5a-550d924c336e', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='LangChain v0.3 | 🦜️🔗 LangChain'),
  Document(id='1601590e-f2d9-4dd4-be2b-0953c91e1778', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='Any package that is now compatible with both langchain-core 0.2 and 0.3 had a patch version bump.\nYou can use the langchain-cli to update deprecated imports automatically.\nThe CLI will handle updating deprecated imports that were introduced in LangChain 0.0.x and LangChain 0.1, as\nwell as updating the langchain_core.pydantic_v1 and langchain.pydantic_v1 imports.\nBase packages\u